In [1]:
%load_ext autoreload
%autoreload 2
import json
import sys
import os
import pandas as pd
from math import floor
import numpy as np
import pickle
sys.path.append(os.path.abspath('../../../utils'))
from protocol_system_compiler import *

In [2]:
fluids = pd.read_csv('fluids.csv')

In [3]:
fluids

,NodeID,FluidType,BufferName,LoadVolume
0,ot2,source,ot2,3
1,2,source,wash buffer,40
2,3,source,imaging buffer,21
3,4,source,tcep,20
4,5,source,2x-SSC,50
5,6,source,universal_fluors,15
6,waste,destination,waste_tank,0


In [4]:
labware = {'tips_200':{
    'api_name': 'opentrons_96_filtertiprack_200ul',
    'location': '1'
},
              'tips_1000':{
    'api_name': 'opentrons_96_filtertiprack_1000ul',
    'location': '2'
              },
              'reservoir1':{
    'api_name': 'nest_12_reservoir_15ml',
    'location': '3'
                },
              'wellplate1':{
    'api_name': 'nest_96_wellplate_2ml_deep',
    'location': '4'
                },
              'tuberack_50ml_1':{
    'api_name': 'opentrons_6_tuberack_falcon_50ml_conical',
    'location': '5'
                },
              'tuberack_15ml_1':{
    'api_name': 'opentrons_10_tuberack_falcon_4x50ml_6x15ml_conical',
    'location': '6'
                },
              'syringe_adapter':{
    'api_name': 'yeolab_1_bd_syringe_adap_3000ul',
    'location': '10'
                },
              }
hardware = {'p300':{
    'api_name': 'p300_single_gen2',
    'mount': 'left',
    'tip_racks': ['tips_200']},
            'p1000':{
    'api_name': 'p1000_single_gen2',
    'mount': 'right',
    'tip_racks': ['tips_1000']}}

metadata = {'apiLevel': '2.14'}

ot2_config = {'labware':labware,
              'hardware':hardware,
              'metadata':metadata}

In [7]:
reagents = {}
"""
reagents = {'wash buffer1':{'labware':'tuberack_50ml_1',
                           'location':'A1',
                           'volume':1000,
                           'hardware;':'p1000',
                           'repeat':2},
            'wash buffer2':{'labware':'tuberack_50ml_1',
                            'location':'A2',
                            'volume':1000,
                           'hardware;':'p1000',
                           'repeat':2},
            'wash buffer3':{'labware':'tuberack_50ml_1',
                            'location':'A3',
                            'volume':1000,
                           'hardware;':'p1000',
                           'repeat':2},
            'wash buffer4':{'labware':'tuberack_50ml_1',
                            'location':'B1',
                            'volume':1000,
                           'hardware;':'p1000',
                           'repeat':2},
            'imaging buffer':{'labware':'tuberack_50ml_1',
                                'location':'B2',
                                'volume':750,
                           'hardware;':'p1000',
                           'repeat':2},
            'tcep':{'labware':'tuberack_50ml_1',
                                'location':'B3',
                                'volume':1000,
                           'hardware;':'p1000',
                           'repeat':2},
            '2x-SSC':{'labware':'tuberack_50ml_1',
                                'location':'A5',
                                'volume':833,
                           'hardware;':'p1000',
                           'repeat':3},
            'universal_fluors':{'labware':'tuberack_15ml_1',
                                'location':'A1',
                                'volume':750,
                           'hardware;':'p1000',
                           'repeat':2}}
"""


"\nreagents = {'wash buffer1':{'labware':'tuberack_50ml_1',\n                           'location':'A1',\n                           'volume':1000,\n                           'hardware;':'p1000',\n                           'repeat':2},\n            'wash buffer2':{'labware':'tuberack_50ml_1',\n                            'location':'A2',\n                            'volume':1000,\n                           'hardware;':'p1000',\n                           'repeat':2},\n            'wash buffer3':{'labware':'tuberack_50ml_1',\n                            'location':'A3',\n                            'volume':1000,\n                           'hardware;':'p1000',\n                           'repeat':2},\n            'wash buffer4':{'labware':'tuberack_50ml_1',\n                            'location':'B1',\n                            'volume':1000,\n                           'hardware;':'p1000',\n                           'repeat':2},\n            'imaging buffer':{'labware':'tubera

In [8]:
def generate_well_pos(well_num,total=96,ncol=12,nrow=8):
    row = well_num//ncol
    col = well_num%ncol
    return chr(65+row)+str(col+1)

In [9]:
num_rounds = 50
for i in range(num_rounds):
    reagents['round'+str(i+1)] = {'labware':'wellplate1',
                                'location':generate_well_pos(i),
                                'volume':750,
                           'hardware':'p1000',
                           'repeat':2}

In [10]:
def get_fluid_node(buffer):
    return fluids[fluids['BufferName']==buffer]['NodeID'].iloc[0]

In [11]:
experiment = {}

In [12]:
step_n = 0

In [13]:
reagents['round6']

{'labware': 'wellplate1',
 'location': 'A6',
 'volume': 750,
 'hardware': 'p1000',
 'repeat': 2}

In [14]:
for round_n in range(1,51):
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'liquid_handler'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['reagent'] = f'round{round_n}'
    experiment[step_n]['step_metadata']['info'] = reagents[f'round{round_n}']
    experiment[step_n]['step_metadata']['destination'] = 'syringe_adapter'
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'fluid'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['fluid'] = get_fluid_node('ot2')
    experiment[step_n]['step_metadata']['volume'] = 1.5
    experiment[step_n]['step_metadata']['speed'] = 1
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'wait'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['wait_time'] = 15*60 # 15 minutes
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'fluid'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['fluid'] = get_fluid_node('wash buffer')
    experiment[step_n]['step_metadata']['volume'] = 2
    experiment[step_n]['step_metadata']['speed'] = 1
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'fluid'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['fluid'] = get_fluid_node('universal_fluors')
    experiment[step_n]['step_metadata']['volume'] = 1.5
    experiment[step_n]['step_metadata']['speed'] = 0.5
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'wait'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['wait_time'] = 15*60 # 15 minutes
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'fluid'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['fluid'] = get_fluid_node('wash buffer')
    experiment[step_n]['step_metadata']['volume'] = 2
    experiment[step_n]['step_metadata']['speed'] = 1
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'fluid'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['fluid'] = get_fluid_node('imaging buffer')
    experiment[step_n]['step_metadata']['volume'] = 1.5
    experiment[step_n]['step_metadata']['speed'] = 0.5
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'wait'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['wait_time'] = 5*60 # 5 minutes
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'image'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['filename'] = f'round_{round_n}'
    experiment[step_n]['step_metadata']['callibrate_af'] = True
    experiment[step_n]['slack_notify'] = True
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'fluid'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['fluid'] = get_fluid_node('tcep')
    experiment[step_n]['step_metadata']['volume'] = 2
    experiment[step_n]['step_metadata']['speed'] = 0.5
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'wait'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['wait_time'] = 5*60 # 5 minutes
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'fluid'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['fluid'] = get_fluid_node('2x-SSC')
    experiment[step_n]['step_metadata']['volume'] = 5
    experiment[step_n]['step_metadata']['speed'] = 1
    experiment[step_n]['slack_notify'] = False
    step_n += 1

In [15]:
experiment

{0: {'step_type': 'liquid_handler',
  'step_metadata': {'reagent': 'round1',
   'info': {'labware': 'wellplate1',
    'location': 'A1',
    'volume': 750,
    'hardware': 'p1000',
    'repeat': 2},
   'destination': 'syringe_adapter'},
  'slack_notify': False},
 1: {'step_type': 'fluid',
  'step_metadata': {'fluid': 'ot2', 'volume': 1.5, 'speed': 1},
  'slack_notify': False},
 2: {'step_type': 'wait',
  'step_metadata': {'wait_time': 900},
  'slack_notify': False},
 3: {'step_type': 'fluid',
  'step_metadata': {'fluid': '2', 'volume': 2, 'speed': 1},
  'slack_notify': False},
 4: {'step_type': 'fluid',
  'step_metadata': {'fluid': '6', 'volume': 1.5, 'speed': 0.5},
  'slack_notify': False},
 5: {'step_type': 'wait',
  'step_metadata': {'wait_time': 900},
  'slack_notify': False},
 6: {'step_type': 'fluid',
  'step_metadata': {'fluid': '2', 'volume': 2, 'speed': 1},
  'slack_notify': False},
 7: {'step_type': 'fluid',
  'step_metadata': {'fluid': '3', 'volume': 1.5, 'speed': 0.5},
  'sl

In [16]:
experiment[646]

{'step_type': 'image',
 'step_metadata': {'filename': 'round_50', 'callibrate_af': True},
 'slack_notify': True}

In [17]:
del experiment[647]
del experiment[648]
del experiment[649]

In [18]:
step_n = 647

In [19]:
experiment[step_n] = {}
experiment[step_n]['step_type'] = 'fluid'
experiment[step_n]['step_metadata'] = {}
experiment[step_n]['step_metadata']['fluid'] = get_fluid_node('2x-SSC')
experiment[step_n]['step_metadata']['volume'] = 5
experiment[step_n]['step_metadata']['speed'] = 1
experiment[step_n]['slack_notify'] = False
step_n += 1

In [20]:
with open('experiment.json', 'w+') as f:
    json.dump(experiment, f, indent=4)


In [21]:
with open('ot2_config.json', 'w+') as f:
    json.dump(ot2_config, f, indent=4)


In [18]:
oni_params = {
    "cropping": {
        "top-left": [130, 5],
        "top-right": [140, 595],
        "width": 428,
        "height": 684
    },
    "safe_focus": {
        "upper_thresh": 2000,
        "lower_thresh": -450
    },
    "microscope_name": "60xAALZ7W2T",
    "xy_positions_mm": {},
    "z_relative_positions_um": [],
    "light_program":[{'laser_power': [10.0, 0.0, 0.0, 0.0], 'exposure_ms': 33}, # ordered 405, 488, 561, 640
                     {'laser_power': [0.0, 0.0, 10.0, 0.0], 'exposure_ms': 100},
                     {'laser_power': [0.0, 0.0, 0.0, 10.0], 'exposure_ms': 100}],
    "start_z": -675.533,
    "num_z_slices": 10,
    "relative_top_z_um": 2.0,
    "relative_bottom_z_um": -2.0,
    'fovIncrement_mm': {'x': 0.08345999908447266,'y': 0.13337998962402345} # ONI default with no overlap
}

def _make_tiles(starting_pos,n_x,n_y,x_inc_um,y_inc_um):
        x_start = starting_pos['x']
        y_start = starting_pos['y']
        positions = []
        xs = list(range(0,n_x))
        ys = list(range(0,n_y))
        if (len(ys) % 2) == 0:
            all_xs = (xs + list(reversed(xs)))*floor(len(ys)/2)
        else:
            all_xs = (xs + list(reversed(xs)))*floor(len(ys)/2)
            all_xs += xs
        all_ys = []
        for y in ys:
            all_ys += [y]*len(xs)
        tmp_positions = list(zip(all_xs,all_ys))
        positions = []
        for pos in tmp_positions:
            positions.append([x_start + (pos[0] * x_inc_um), y_start + (pos[1] * y_inc_um)])
        return positions

# custom_positions = None
def position_generator(oni_params,custom_positions,starting_xy_um={'x':0,'y':0},tile_num_x=1,tile_num_y=1,overlap=0.0):
    if custom_positions is not None:
        oni_params['xy_positions_mm'] = custom_positions
    else:
        x_inc_um = (oni_params['fovIncrement_mm']['x'] * 1000) * (1 - overlap)
        y_inc_um = (oni_params['fovIncrement_mm']['y'] * 1000) * (1 - overlap)
        oni_params['xy_positions_mm'] = _make_tiles(starting_xy_um,tile_num_x,tile_num_y,x_inc_um,y_inc_um)
    # generate relative_zs
    oni_params['z_relative_positions_um'] = np.linspace(oni_params['relative_top_z_um'],oni_params['relative_bottom_z_um'],oni_params['num_z_slices']).tolist()
    return oni_params

In [21]:
oni_params = position_generator(oni_params,None,{'x':5.25*1000,'y':2*1000},10,5,0.0)

In [22]:
with open('oni_params.pkl', 'wb') as f:
    pickle.dump(oni_params, f)

In [26]:
fluids

,NodeID,FluidType,BufferName,LoadVolume
0,ot2,source,ot2,3
1,2,source,wash buffer,40
2,3,source,imaging buffer,21
3,4,source,tcep,20
4,5,source,2x-SSC,50
5,6,source,universal_fluors,15
6,waste,destination,waste_tank,0


In [29]:
def get_fluid_name(nodeID):
    return fluids[fluids['NodeID']==nodeID]['BufferName'].iloc[0]

In [47]:
reagent_vols = {}
for i in experiment.keys():
    if experiment[i]['step_type'] == 'fluid':
        buffer = get_fluid_name(experiment[i]['step_metadata']['fluid'])
        if buffer not in reagent_vols:
            reagent_vols[buffer] = 0
        reagent_vols[buffer] += experiment[i]['step_metadata']['volume']

In [48]:
reagent_vols

{'ot2': 75.0,
 'wash buffer': 200,
 'universal_fluors': 75.0,
 'imaging buffer': 75.0,
 'tcep': 98,
 '2x-SSC': 250}

In [49]:
for i in experiment.keys():
    if experiment[i]['step_type'] == 'liquid_handler':
        print(experiment[i]['step_metadata']['info']['location'])

A1
A2
A3
A4
A5
A6
A7
A8
A9
A10
A11
A12
B1
B2
B3
B4
B5
B6
B7
B8
B9
B10
B11
B12
C1
C2
C3
C4
C5
C6
C7
C8
C9
C10
C11
C12
D1
D2
D3
D4
D5
D6
D7
D8
D9
D10
D11
D12
E1
E2
